<a href="https://colab.research.google.com/github/0x1beef/uap/blob/main/nb/gimbal_adjust_clouds.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/0x1beef/uap/blob/main/nb/gimbal_adjust_clouds.ipynb">
    <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"   />
</a>

## **Get the data**

In [ ]:
url = 'https://raw.githubusercontent.com/0x1beef/uap/main/nb'
import urllib.request
for py_file in ['utils.py','common.py']:
    urllib.request.urlretrieve(f'{url}/{py_file}', py_file)
import utils, common

In [ ]:
utils.download_from_huggingface('logicbear/gimbal/data/object_data.parquet')
utils.download_from_huggingface('logicbear/gimbal_clouds/data/cloud_motion.parquet')

In [ ]:
(object_data, object_metadata) = utils.from_parquet_ext('data/object_data.parquet', 'gimbal')
(cloud_data, cloud_metadata) = utils.from_parquet_ext('data/cloud_motion.parquet', 'clouds')

params = cloud_metadata['params']
print(f'frame_diff: {params["frame_diff_from"]}-{params["frame_diff_to"]}')

import numpy as np

def get_com(frame):
    od = object_data.loc[frame]
    return np.array([od.center_of_mass_x, od.center_of_mass_y]).T

def get_horizon(frame):
    return object_data.loc[frame].human_horizon

## **Adjust to account for background motion/rotation**

In [ ]:
import math
from dataclasses import dataclass
from numba import njit

@dataclass
class AdjustedMotion:
    frame: int
    magnitude: float; angle: float; rotation: float

@njit
def inlier_mean(corners, inliers):
    (sum_x, sum_y, num_inliers) = (0.0, 0.0, 0)
    for ((cx, cy), inlier) in zip(corners, inliers):
        if inlier:
            sum_x += float(cx); sum_y += float(cy)
            num_inliers += 1
    return np.array([sum_x / num_inliers, sum_y / num_inliers]).T

def adjust_cloud_motion(frame):
    cd = cloud_data.loc[frame]
    R = cd['transform'][0:2,0:2]
    scale = math.sqrt(R[0,0]*R[0,0] + R[0,1]*R[0,1])
    R = R / scale
    C = np.array([213,211]).T
    P1 = inlier_mean(cd.corners, cd.inliers)
    P2 = inlier_mean(cd.corners_next, cd.inliers)
    Rinv = np.linalg.inv(R)
    O1 = get_com(frame)
    O2 = get_com(frame + cd.frame_diff)
    V = Rinv @ (P2 - C) - P1 + C - (O2 - O1)
    magnitude = np.linalg.norm(V) / cd.frame_diff
    angle = math.degrees(math.atan2(V[1], V[0]))
    rotation = cd.rotation + (get_horizon(frame + cd.frame_diff) - get_horizon(frame))
    return AdjustedMotion(frame, magnitude, angle, rotation)

def adjust_cloud_motion_for(frames):
    return [adjust_cloud_motion(frame) for frame in frames]
def run_adjust_cloud_motion():
    return adjust_cloud_motion_for(range(len(cloud_data)))
def run_adjust_cloud_motion_parallel():
    return utils.run_jobs_in_parallel(thread_func = adjust_cloud_motion_for,
        jobs = range(len(cloud_data)), threads = 4)

adjust_cloud_motion(0)

In [ ]:
%%time
adj_motions = run_adjust_cloud_motion() #_parallel()
print(len(adj_motions))

## **Plot the results**

In [ ]:
import pandas as pd
df = pd.DataFrame.from_records([m.__dict__ for m in adj_motions], index = ["frame"])
df = df.sort_index()

df = common.gimbal_fix_wh_to_bh(df, ['magnitude','angle','rotation'])

import matplotlib.pyplot as plt

def plot(plot_func, x_range):
    plot_func(x_range)
    ax = plt.gca()
    ax.set_xlabel('frames',loc='right')
    fps = object_metadata['fps']
    ax2 = ax.secondary_xaxis(location='top', 
        functions=(lambda frame: frame / fps, lambda time: time * fps))
    ax2.set_xlabel('time [s]',loc='right')
    plt.grid()
    plt.legend()
    plt.show()

def mag_plot(range):
    plt.title('cloud motion magnitude')
    plt.ylabel('pixels / frame')
    cloud_data.magnitude[range].plot(color='gray')
    df.magnitude[range].plot(color='orange', label='mag adjusted')

plot(mag_plot, range(len(cloud_data)))
plot(mag_plot, range(700, len(cloud_data)))

def angle_plot(range):
    plt.title('cloud motion angle')
    plt.ylabel('degrees')
    cloud_data.angle[range].plot(color='gray')
    df.angle[range].plot(color='orange', label='angle adjusted')
    plt.ylim(bottom=13, top=48)
    (-object_data.human_horizon)[range].plot()

plot(angle_plot, range(len(cloud_data)))

def rot_plot(range):
    plt.title('cloud rotation')
    plt.ylabel('degrees')
    cloud_data.rotation[range].plot(color='gray')
    df.rotation[range].plot(color='orange', label='rot adjusted')

plot(rot_plot, range(len(cloud_data)))
plot(rot_plot, range(700, len(cloud_data)))